### to be continued... ->> I moved the cnn-modeling to a cnn-images.py file
- am more than happy to assist someone in transfering it back here if you guys prefer notebooks over scripts

In [ ]:
# System and utilities
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['OMP_NUM_THREADS'] = '12'
os.environ['TF_NUM_INTEROP_THREADS'] = '4'
os.environ['TF_NUM_INTRAOP_THREADS'] = '12'
import time
import argparse
import json
# Data manipulation and analysis

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import sys
sys.path.append('../src/functions')
from training_report import get_training_report


## KERAS
from tensorflow.keras.utils import set_random_seed
set_random_seed(66)  # Set random seed for reproducibility
from tensorflow.config.threading import set_intra_op_parallelism_threads, set_inter_op_parallelism_threads
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore")


In [ ]:
N_EPOCHS          = 100
BATCH_SIZE        = 64
LR                = 0.01
DATASET_PERC      = 0.9
IMG_SIZE          = 224## 224 minimum
CONV_FILTERS_1, CONV_FILTERS_2, CONV_FILTERS_3, CONV_FILTERS_4, CONV_FILTERS_5 = 16,16,32,64,128
CONV_K_REG        = 0.0001
DENSE_K_REG       = 0.001

In [ ]:
# LOAD DATA:
OUT_PATH = '../misc/'
PROC_DATA_PATH = '../processed_data/'
df_full = pd.read_csv(PROC_DATA_PATH + 'X_train_multimodal.csv')
df = df_full.head(int(df_full.shape[0]*DATASET_PERC))

##### 1. Create string labels for generators
##### 2. Split data 
##### 3. Compute class weights

In [ ]:
# Create string labels for generators
df['prdtypecode_str'] = df['prdtypecode'].astype(str)

# Split data 
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=66, 
    stratify=df['prdtypecode'])

# Compute class weights 
y_train_for_weights = train_df['prdtypecode'].astype('category').cat.codes
u_classes = np.unique(y_train_for_weights)
class_weights = compute_class_weight('balanced', classes=u_classes, y=y_train_for_weights)
class_weight_dict = {int(cls): round(float(weight), 3) for cls, weight in zip(u_classes, class_weights)}
print(f'Class weights: {class_weight_dict}')

##### Definition of data augmentation generators

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

##### Create data generators that load images from disk --> and apply data augmentation using the generators from the cell above

In [ ]:
   
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='imagepath',
    y_col='prdtypecode_str',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=66)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='imagepath',
    y_col='prdtypecode_str',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=66)

# BUILDING THE CNN MODEL

In [ ]:
### MODEL SETUP:
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# C1
conv1 = Conv2D(filters=CONV_FILTERS_1, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(input_layer)
conv1 = BatchNormalization()(conv1)
conv1 = Conv2D(filters=CONV_FILTERS_1, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(conv1)
conv1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
pool1 = Dropout(0.25)(pool1)
# C2
conv2 = Conv2D(filters=CONV_FILTERS_2, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(pool1)
conv2 = BatchNormalization()(conv2)
conv2 = Conv2D(filters=CONV_FILTERS_2, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(conv2)
conv2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
pool2 = Dropout(0.25)(pool2)
# C3
conv3 = Conv2D(filters=CONV_FILTERS_3, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(pool2)
conv3 = BatchNormalization()(conv3)
conv3 = Conv2D(filters=CONV_FILTERS_3, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(conv3)
conv3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
pool3 = Dropout(0.25)(pool3)
# C4
conv4 = Conv2D(filters=CONV_FILTERS_4, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(pool3)
conv4 = BatchNormalization()(conv4)
conv4 = Conv2D(filters=CONV_FILTERS_4, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(conv4)
conv4 = BatchNormalization()(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
pool4 = Dropout(0.25)(pool4)
# # C5
conv5 = Conv2D(filters=CONV_FILTERS_5, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(pool4)
conv5 = BatchNormalization()(conv5)
conv5 = Conv2D(filters=CONV_FILTERS_5, kernel_size=(3, 3), 
                activation='relu', kernel_regularizer=l2(CONV_K_REG))(conv5)
conv5 = BatchNormalization()(conv5)
pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
pool5 = Dropout(0.25)(pool5)

# Global Average Pooling
gap = GlobalAveragePooling2D()(pool5) ## change this to pool4 or pool5 if you want to use deeper layers

# Dense layers
# D1
dense1 = Dense(512, activation='relu', kernel_regularizer=l2(DENSE_K_REG))(gap)
dense1 = BatchNormalization()(dense1)
dense1 = Dropout(0.4)(dense1)
# D2
dense2 = Dense(256, activation='relu', kernel_regularizer=l2(DENSE_K_REG))(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Dropout(0.3)(dense2)

# D3
dense3 = Dense(64, activation='relu', kernel_regularizer=l2(DENSE_K_REG))(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Dropout(0.3)(dense3)

# Output layer
output = Dense(len(df['prdtypecode'].unique()), activation='softmax', kernel_regularizer=l2(0.001))(dense3)

model = Model(inputs=input_layer, outputs=output)

##### Define the metric, early stopping, and two options for learning rate control

In [ ]:
f1_metric = F1Score(average='macro', name='f1_score')
early_stopping = EarlyStopping(monitor='val_f1_score', patience=7, restore_best_weights=True, verbose=1, mode='max')

# Option 1: Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1, mode='min')
# Option 2: Cosine decay with restarts
lr_schedule = CosineDecayRestarts(
initial_learning_rate=LR,
first_decay_steps=10,        # First cycle length
t_mul=1.5,                   # Each cycle gets 1.5x longer
m_mul=0.6,                   # LR drops 40% after each restart
alpha=0.000001                 # Minimum LR
)

##### Seeing the full setup

In [ ]:
model.compile(optimizer=AdamW(learning_rate=LR, weight_decay=0.01), loss='categorical_crossentropy',
                metrics=['accuracy', f1_metric])
print(model.summary())

print('--'* 50)
print(f'Using {DATASET_PERC*100}% of the dataset for training ({int(df_full.shape[0]*DATASET_PERC)} rows) .')
print(f'Training set: {len(train_df)} samples, Validation set: {len(val_df)} samples')
print(f'Starting Image CNN training with parameters:')
print(f'------>number of epochs={N_EPOCHS}, batch_size={BATCH_SIZE}, initial learning_rate={LR}') 
print(f'------>image_size={IMG_SIZE}x{IMG_SIZE}')
print(f'------>conv_filters=[{CONV_FILTERS_1, CONV_FILTERS_2,CONV_FILTERS_3,CONV_FILTERS_4,CONV_FILTERS_5}], conv_k_reg={CONV_K_REG}, dense_k_reg={DENSE_K_REG}')
print('--'* 50)

In [ ]:
# LONG RUNTIME WARNING !!!
#### Training the model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=N_EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict,
    verbose=1
)
print('Image CNN training completed!')

In [ ]:
model_name = f'image-cnn-epochs-{N_EPOCHS}-lr-{LR}_testing_valf1-{history.history["val_f1_score"][-1]:.3f}'

In [ ]:
### Getting the training report

In [ ]:
import datetime
import seaborn as sns

In [ ]:
validation_data = val_generator
    
# Use Keras built-in evaluate
val_metrics = model.evaluate(validation_data, verbose=1)
metric_names = model.metrics_names
val_results = dict(zip(metric_names, val_metrics))

# Generate predictions
print("Generating predictions...")
validation_data.reset()
y_pred = model.predict(validation_data, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_data.classes


class_names = df.prdtypecode_str.unique().tolist()  # Get class names from the DataFrame
# Get class names and metadata
if class_names is None:
    class_names = list(validation_data.class_indices.keys())

batch_size = validation_data.batch_size
input_info = f"Image Size: {validation_data.target_size}"


In [ ]:
# Calculate additional metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

if 'f1_score' in history.history:
    final_train_f1 = history.history['f1_score'][-1]
    final_val_f1 = history.history['val_f1_score'][-1]
else:
    final_train_f1 = final_val_f1 = "N/A"

# Generate classification report
class_report = classification_report(y_true, y_pred_classes, 
                                    target_names=class_names, 
                                    output_dict=True)

In [ ]:
history.history

In [ ]:
# TRAINING CURVES + SUMMARY
fig = plt.figure(figsize=(11, 8.5))

# Create grid: 2x3 layout
gs = fig.add_gridspec(2, 3, height_ratios=[1, 1], width_ratios=[1, 1, 0.8])

# Training curves (top row)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(history.history['loss'], label='Train', linewidth=2)
ax1.plot(history.history['val_loss'], label='Val', linewidth=2)
ax1.set_title('Loss', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(history.history['accuracy'], label='Train', linewidth=2)
ax2.plot(history.history['val_accuracy'], label='Val', linewidth=2)
ax2.set_title('Accuracy', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# F1 and LR (bottom row)
ax3 = fig.add_subplot(gs[1, 0])
if 'f1_score' in history.history:
    ax3.plot(history.history['f1_score'], label='Train', linewidth=2)
    ax3.plot(history.history['val_f1_score'], label='Val', linewidth=2)
    ax3.set_title('F1 Score', fontweight='bold')
    ax3.legend()
else:
    ax3.text(0.5, 0.5, 'F1 not tracked', ha='center', va='center')
    ax3.set_title('F1 Score (N/A)')
ax3.grid(True, alpha=0.3)

ax4 = fig.add_subplot(gs[1, 1])
if 'learning_rate' in history.history:
    ax4.plot(history.history['learning_rate'], linewidth=2, color='red')
    ax4.set_title('Learning Rate', fontweight='bold')
    ax4.set_yscale('log')
else:
    ax4.text(0.5, 0.5, 'LR not tracked', ha='center', va='center')
    ax4.set_title('Learning Rate (N/A)')
ax4.grid(True, alpha=0.3)

# Summary text (right side)
ax_text = fig.add_subplot(gs[:, 2])
ax_text.axis('off')

# Format F1 scores safely
train_f1_str = f"{final_train_f1:.3f}" if final_train_f1 != "N/A" else "N/A"
val_f1_str = f"{final_val_f1:.3f}" if final_val_f1 != "N/A" else "N/A"

summary_text = f"""{model_name}

FINAL METRICS
Train Acc: {final_train_acc:.3f}
Val Acc:   {final_val_acc:.3f}
Train F1:  {train_f1_str}
Val F1:    {val_f1_str}

Macro F1:  {class_report['macro avg']['f1-score']:.3f}
Weight F1: {class_report['weighted avg']['f1-score']:.3f}

MODEL INFO
Params: {model.count_params():,}
Epochs: {len(history.history['loss'])}
Classes: {len(class_names)}
{input_info}"""

ax_text.text(0.05, 0.95, summary_text, transform=ax_text.transAxes, 
            fontsize=9, verticalalignment='top', fontfamily='monospace')

plt.suptitle(f'Training Report - {model_name}', fontsize=14, fontweight='bold')
plt.tight_layout()


In [ ]:

# PAGE 2: CONFUSION MATRIX + CLASS METRICS
fig = plt.figure(figsize=(11, 8.5))
gs = fig.add_gridspec(2, 2, height_ratios=[1.2, 1])

# Confusion matrix (top, spanning both columns)
ax_cm = fig.add_subplot(gs[0, :])
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names, ax=ax_cm)
ax_cm.set_title('Confusion Matrix', fontweight='bold')
ax_cm.set_xlabel('Predicted')
ax_cm.set_ylabel('Actual')

# Class metrics (bottom row)
report_df = pd.DataFrame(class_report).transpose()
class_metrics = report_df.loc[class_names, ['precision', 'recall', 'f1-score']]

# Top 10 and bottom 10 classes by F1
sorted_by_f1 = class_metrics.sort_values('f1-score')
worst_10 = sorted_by_f1.head(10)
best_10 = sorted_by_f1.tail(10)

ax_worst = fig.add_subplot(gs[1, 0])
worst_10['f1-score'].plot(kind='barh', ax=ax_worst, color='lightcoral')
ax_worst.set_title('Bottom 10 Classes (F1 Score)', fontweight='bold')
ax_worst.set_xlabel('F1 Score')

ax_best = fig.add_subplot(gs[1, 1])
best_10['f1-score'].plot(kind='barh', ax=ax_best, color='lightgreen')
ax_best.set_title('Top 10 Classes (F1 Score)', fontweight='bold')
ax_best.set_xlabel('F1 Score')

plt.tight_layout()

In [ ]:
summary = {
    'Model Name': model_name,

    'Total Parameters': model.count_params(),
    'Training Epochs': len(history.history['loss']),
    'Final Training Accuracy': f"{final_train_acc:.4f}",
    'Final Validation Accuracy': f"{final_val_acc:.4f}",
    'Final Training Loss': f"{final_train_loss:.4f}",
    'Final Validation Loss': f"{final_val_loss:.4f}",
    'Final Training F1': f"{final_train_f1:.4f}" if final_train_f1 != "N/A" else "N/A",
    'Final Validation F1': f"{final_val_f1:.4f}" if final_val_f1 != "N/A" else "N/A",
    'Validation Metrics': val_results,
    'Macro Avg F1': f"{class_report['macro avg']['f1-score']:.4f}",
    'Weighted Avg F1': f"{class_report['weighted avg']['f1-score']:.4f}",
    'Number of Classes': len(class_names),
    'Batch Size': batch_size,
    'Input Info': input_info,
}

print(summary)